In [20]:
# libraries
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder


In [3]:
dir = os.getcwd()
file_path = os.path.join(dir,"DataSets-main" )
cab_df = pd.read_csv(file_path + "/Cab_Data.csv")
city_df = pd.read_csv(file_path + "/City.csv")
customer_df = pd.read_csv(file_path + "/Customer_ID.csv")
transaction_df = pd.read_csv(file_path + "/Transaction_ID.csv")


cab_df.head()

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
0,10000011,42377,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,42375,Pink Cab,ATLANTA GA,28.62,358.52,334.854
2,10000013,42371,Pink Cab,ATLANTA GA,9.04,125.20,97.632
3,10000014,42376,Pink Cab,ATLANTA GA,33.17,377.40,351.602
4,10000015,42372,Pink Cab,ATLANTA GA,8.73,114.62,97.776


In [4]:
# Get information of the cab dataframe
cab_df.info()
# Percentage of missing values
print("Percentage of missing values")
cab_df.isnull().sum()/ len(cab_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359392 entries, 0 to 359391
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Transaction ID  359392 non-null  int64  
 1   Date of Travel  359392 non-null  int64  
 2   Company         359392 non-null  object 
 3   City            359392 non-null  object 
 4   KM Travelled    359392 non-null  float64
 5   Price Charged   359392 non-null  float64
 6   Cost of Trip    359392 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 19.2+ MB
Percentage of missing values


Transaction ID    0.0
Date of Travel    0.0
Company           0.0
City              0.0
KM Travelled      0.0
Price Charged     0.0
Cost of Trip      0.0
dtype: float64

In [5]:
city_df.head()


,City,Population,Users
0,NEW YORK NY,"8,405,837","302,149"
1,CHICAGO IL,"1,955,130","164,468"
2,LOS ANGELES CA,"1,595,037","144,132"
3,MIAMI FL,"1,339,155","17,675"
4,SILICON VALLEY,"1,177,609","27,247"


In [6]:
city_df.info()

#Checking for null values
city_df.isna().sum() / len(city_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   City        20 non-null     object
 1   Population  20 non-null     object
 2   Users       20 non-null     object
dtypes: object(3)
memory usage: 612.0+ bytes


City          0.0
Population    0.0
Users         0.0
dtype: float64

In [7]:
customer_df.head()


,Customer ID,Gender,Age,Income (USD/Month)
0,29290,Male,28,10813
1,27703,Male,27,9237
2,28712,Male,53,11242
3,28020,Male,23,23327
4,27182,Male,33,8536


In [8]:
customer_df.info()

customer_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49171 entries, 0 to 49170
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer ID         49171 non-null  int64 
 1   Gender              49171 non-null  object
 2   Age                 49171 non-null  int64 
 3   Income (USD/Month)  49171 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


Customer ID           0
Gender                0
Age                   0
Income (USD/Month)    0
dtype: int64

In [9]:
transaction_df.head()

,Transaction ID,Customer ID,Payment_Mode
0,10000011,29290,Card
1,10000012,27703,Card
2,10000013,28712,Cash
3,10000014,28020,Cash
4,10000015,27182,Card


In [10]:
transaction_df.info()

transaction_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440098 entries, 0 to 440097
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Transaction ID  440098 non-null  int64 
 1   Customer ID     440098 non-null  int64 
 2   Payment_Mode    440098 non-null  object
dtypes: int64(2), object(1)
memory usage: 10.1+ MB


Transaction ID    0
Customer ID       0
Payment_Mode      0
dtype: int64

## Combining the datasets to one dataframe

In [11]:
# Customer information
customer_info = pd.merge(customer_df, transaction_df)
customer_info.head(30)


,Customer ID,Gender,Age,Income (USD/Month),Transaction ID,Payment_Mode
0,29290,Male,28,10813,10000011,Card
1,29290,Male,28,10813,10351127,Cash
2,29290,Male,28,10813,10412921,Card
3,27703,Male,27,9237,10000012,Card
4,27703,Male,27,9237,10320494,Card
5,27703,Male,27,9237,10324737,Cash
6,27703,Male,27,9237,10395626,Card
7,28712,Male,53,11242,10000013,Cash
8,28712,Male,53,11242,10079404,Card
9,28712,Male,53,11242,10186994,Card


In [12]:
# Cab information
cab_info = pd.merge(cab_df, city_df)
cab_info.head()

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip,Population,Users
0,10000011,42377,Pink Cab,ATLANTA GA,30.45,370.95,313.635,"814,885","24,701"
1,10000012,42375,Pink Cab,ATLANTA GA,28.62,358.52,334.854,"814,885","24,701"
2,10000013,42371,Pink Cab,ATLANTA GA,9.04,125.20,97.632,"814,885","24,701"
3,10000014,42376,Pink Cab,ATLANTA GA,33.17,377.40,351.602,"814,885","24,701"
4,10000015,42372,Pink Cab,ATLANTA GA,8.73,114.62,97.776,"814,885","24,701"


In [13]:
customer_info.columns

Index(['Customer ID', 'Gender', 'Age', 'Income (USD/Month)', 'Transaction ID',
       'Payment_Mode'],
      dtype='object')

In [14]:
#print the shapes of the initial dataframes
print(f"Shape for cab: {cab_df.shape}, customer : {customer_df.shape}, City: {city_df.shape}, transaction: {transaction_df.shape}")

Shape for cab: (359392, 7), customer : (49171, 4), City: (20, 3), transaction: (440098, 3)


In [15]:
customer_info.to_csv("customer_information.csv")
cab_info.to_csv("cab_information.csv")

In [24]:
#HotEncoding the categorical columns
one = OneHotEncoder()
encoded = one.fit_transform([customer_info["Gender"]])
print(encoded)

<1x440098 sparse matrix of type '<class 'numpy.float64'>'
	with 440098 stored elements in Compressed Sparse Row format>

## Understanging the relationships between the data

In [18]:
correlation = customer_info.corr()

/tmp/ipykernel_21061/2092938106.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = customer_info.corr()


In [19]:
correlation.style.background_gradient (cmap = 'BrBG')

,Customer ID,Age,Income (USD/Month),Transaction ID
Customer ID,1.000000,-0.002161,-0.005834,-0.021289
Age,-0.002161,1.000000,-0.000573,-0.001060
Income (USD/Month),-0.005834,-0.000573,1.000000,-0.000935
Transaction ID,-0.021289,-0.001060,-0.000935,1.000000
